In [1]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
using JPEC, Plots

  Activating project at `~/Projects/Perturbed-Equilibrium/JPEC.worktrees/vacuum_julia`
Precompiling project...
   4615.6 ms  ✓ JPEC
  1 dependency successfully precompiled in 6 seconds. 181 already precompiled.
  1 dependency had output during precompilation:
┌ JPEC
│  gfortran -fPIC -frecursive -c defs.f -o defs.o
│  gfortran -fPIC -frecursive -c spline.f -o spline.o
│  gfortran -fPIC -frecursive -c cspline.f -o cspline.o
│  gfortran -fPIC -frecursive -c bicube.f -o bicube.o
│  gfortran -fPIC -frecursive -c spline_c_api.f -o spline_c_api.o
│  gfortran -shared -o libspline.dylib defs.o spline.o cspline.o bicube.o spline_c_api.o -framework Accelerate
│  mv libspline.dylib ../../../deps/
│  [ Info: Splines-fortran compiled well
│  gfortran -fPIC -finit-local-zero -frecursive -std=legacy -c vacuum_global.f -o vacuum_global.o
│  cc    -c -o bzio.o bzio.c
│  gfortran -fPIC -finit-local-zero -frecursive -std=legacy -c aleg_new.f -o aleg_new.o
│  gfortran -fPIC -finit-local-zero -frecursive -

In [2]:
function fouran!(gij::Matrix{Float64}, gil::Matrix{Float64},
                 cs::Matrix{Float64}, m00::Int, l00::Int,
                 lmin::Vector{Int}, lmax::Vector{Int}, mth::Int)

    jmax1 = lmax[1] - lmin[1] + 1

    for l1 in 1:jmax1
        coeffs = cs[1:mth, l1]        # vector of Fourier coefficients
        gil[m00+1:m00+mth, l00+l1] .= gij[1:mth, 1:mth] * coeffs
    end

    return nothing
end

fouran! (generic function with 1 method)

In [3]:
mth   = 4
lmin  = [1]
lmax  = [3]
m00   = 0
l00   = 0

gij = rand(mth, mth)
cs  = rand(mth, lmax[1]-lmin[1]+1)
gil = zeros(mth+5, lmax[1]-lmin[1]+1+5)

fouran!(gij, gil, cs, m00, l00, lmin, lmax, mth)

In [4]:
println("gil = ")
println(gil)              # prints entire matrix

gil = 
[1.2370464504822327 0.9710577475774792 1.6605253351633293 0.0 0.0 0.0 0.0 0.0; 1.401412366543789 1.0931911451284595 1.587421851433778 0.0 0.0 0.0 0.0 0.0; 1.0040394857217434 0.7916542262236212 1.5582565368976629 0.0 0.0 0.0 0.0 0.0; 1.286034191514505 1.0122538611854093 1.4101994842108636 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


In [8]:
function foranv!(gil::Array{Float64,2}, gll::Array{Float64,2}, cs::Array{Float64,2},
                 m00::Int, l00::Int, mth::Int, lmin::Int, lmax::Int,
                 dth::Float64, twopi::Float64)

    jmax1 = lmax - lmin + 1

    # Zero out the block of gll that will be filled
    for l1 in 1:jmax1
        for l2 in 1:jmax1
            gll[l2,l1] = 0.0
        end
    end

    # Main summation
    for l1 in 1:jmax1
        for l2 in 1:jmax1
            for i in 1:mth
                gll[l2,l1] += dth * cs[i,l2] * gil[m00+i, l00+l1] * twopi
            end
        end
    end

    return gll
end

foranv! (generic function with 1 method)

In [9]:
function test_foranv()
    # Example parameters (adjust to match your case)
    mth   = 4        # number of poloidal points
    lmin  = 1
    lmax  = 3
    m00   = 0
    l00   = 0
    dth   = 0.1
    twopi = 2π

    nths2, nfm2 = 8, 8  # dimensions for gil
    nfm         = 4     # dimension for cs and gll

    # Dummy input data
    gil = randn(nths2, nfm2)
    cs  = randn(mth, nfm)
    gll = zeros(nfm, nfm)

    # Run the translation
    result = foranv!(gil, gll, cs, m00, l00, mth, lmin, lmax, dth, twopi)

    println("gil =")
    println(gil)
    println("\ncs =")
    println(cs)
    println("\ngll result =")
    println(result)

    return result
end

# Run test
test_foranv()


gil =
[-0.03452597379267958 0.18542481155615056 0.3489552341269621 -1.6302928838910598 0.1970510381999457 -0.14839230883842439 0.6868958872740458 0.8018696175202392; 0.21234697522738297 0.36521820041418884 -1.1389408929473268 -0.644368133270796 1.5422434589276723 0.5224457644864959 -0.3472448121838069 1.820327980622333; -1.7926704402682423 -0.4723779434940146 -1.0521827337912817 0.7243418043370197 0.2623182743792517 -0.6013527382565595 -0.6107343232713054 -0.07842667121856375; -0.02504098322377936 -0.0978845520940032 0.3274843473694329 1.746540800846818 1.6919974041613146 -1.051818337454954 -0.4005815980107553 1.7568263771400678; 1.7330979635266064 -0.7636319618264513 -1.7013420120354399 -0.6031823388340101 -1.4555767071288348 1.6096792755805884 -0.006406050108533869 0.1708537206125385; 2.8172009264222604 0.16741026442238352 0.9695025466518253 0.519654124893962 -0.2347661032771603 0.2582513894925455 0.32102772053953793 0.7969326053065734; 1.4328325918979419 0.04917604446312788 0.247276

4×4 Matrix{Float64}:
  1.93791    0.830273  -0.0889346  0.0
  1.1588     0.142976   1.19481    0.0
 -0.320243  -0.19644   -0.241595   0.0
  0.0        0.0        0.0        0.0